## Optimization of Staffing Levels

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import time 
import pandas as pd 
import numpy as np

In [ ]:
def labor_scheduling(df_index, demand, MaxStaff, patients_staff, availability, MinSL, TargetSL):
    # Creating model instance
    model = pyo.ConcreteModel()

    #### Sets ####
    model.setShift = pyo.Set(initialize=df_index)



    #### Parameters ####
    
    # Demand of Patients
    model.D = pyo.Param(model.setShift, initialize = demand, within = pyo.Integers)
    D = model.D

    # Maximum capacity of staff for each shift
    model.MaxStaff = pyo.Param(initialize = MaxStaff, within = pyo.Integers)
    Cap = model.MaxStaff

    # Number of patients each staff can handle at the same time
    model.PatientsPerStaff = pyo.Param(initialize = patients_staff, within = pyo.Integers)
    PPS = model.PatientsPerStaff

    # Available staff at the time of an occuring shift
    model.AvailableStaff = pyo.Param(model.setShift, initialize = availability, within = pyo.Integers)
    A = model.AvailableStaff

    # Minimum required Service Level
    model.MinServiceLevel = pyo.Param(initialize = MinSL, within = pyo.PercentFraction)
    SL = model.MinServiceLevel

    # Target Service Level
    model.TargetServiceLevel = pyo.Param(initialize = TargetSL, within = pyo.PercentFraction)
    TL = model.TargetServiceLevel



    #### Decision Variables ####
    
    # Number of staff members allocated to a shift
    model.Staff_Allocated = pyo.Var(model.setShift, within = pyo.Integers, bounds=(0.0, None))
    x = model.Staff_Allocated



    #### Objective Function ####

    # Minimize Staffing Levels
    model.obj_min_staff = pyo.Objective(expr = x, sense = pyo.minimize)



    #### Constraints ####

    # Demand Constraint
    model.C1 = pyo.ConstraintList()
    for s in model.setShift:
        model.C1.add(expr = x[s] >= (D[s]/PPS))

    # Staff Availability
    model.C2 = pyo.ConstraintList()
    for s in model.setShift:
        model.C2.add(expr = x[s] <= A[s])

    # Capacity
    model.C3 = pyo.ConstraintList()
    for s in model.setShift:
        model.C3.add(expr = x[s] <= Cap)
    
    # Minimum Service Level
    model.C4 = pyo.ConstraintList()
    for s in model.setShift:
        model.C4.add(expr = (x[s]*PPS)/D[s] >= SL)
    
    # Target Service Level
    model.C5 = pyo.ConstraintList()
    for s in model.setShift:
        model.C5.add(expr = (x[s]*PPS)/D[s] == TL)

    return model


def optimize_staffing(model):
    opt = SolverFactory('glpk')
    status = opt.solve(model, tee=False)
    return model, status